Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [1]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader) 

enrollments = read_csv('enrollments.csv')

print(enrollments[0])

{'cancel_date': '2015-01-14', 'join_date': '2014-11-10', 'is_canceled': 'True', 'days_to_cancel': '65', 'status': 'canceled', 'account_key': '448', 'is_udacity': 'True'}


In [2]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.
daily_engagement = read_csv('daily_engagement.csv')

project_submissions = read_csv('project_submissions.csv')

print(daily_engagement[0])

{'projects_completed': '0.0', 'utc_date': '2015-01-09', 'total_minutes_visited': '11.6793745', 'lessons_completed': '0.0', 'acct': '0', 'num_courses_visited': '1.0'}


## Fixing Data Types

In [3]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{'account_key': '448',
 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 'days_to_cancel': 65,
 'is_canceled': True,
 'is_udacity': True,
 'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 'status': 'canceled'}

In [4]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".
for a in daily_engagement:
    a['account_key'] = a['acct']
    del a['acct']
    
daily_engagement[0]

{'account_key': '0',
 'lessons_completed': 0,
 'num_courses_visited': 1,
 'projects_completed': 0,
 'total_minutes_visited': 11.6793745,
 'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [5]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{'account_key': '256',
 'assigned_rating': 'UNGRADED',
 'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [6]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

def get_unique_students(inVar):
    unique_students = set()
    for a in inVar:
        unique_students.add(a['account_key'])
    return unique_students

enrollment_num_rows = len(enrollments)
enrollment_num_unique_students = len(get_unique_students(enrollments))

engagement_num_rows = len(daily_engagement)
unique_engagements = get_unique_students(daily_engagement)
engagement_num_unique_students = len(unique_engagements)

submission_num_rows = len(project_submissions)
submission_num_unique_students = len(get_unique_students(project_submissions))

print(enrollment_num_unique_students)
print(engagement_num_unique_students)
print(submission_num_unique_students)

1302
1237
743


## Problems in the Data

In [7]:
print(daily_engagement[0]['account_key'])

0


## Missing Engagement Records

In [8]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.

for a in enrollments:
    if a['account_key'] not in unique_engagements:
        print(a)
        break

{'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'is_canceled': True, 'days_to_cancel': 0, 'status': 'canceled', 'account_key': '1219', 'is_udacity': False}


## Checking for More Problem Records

In [9]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.

for a in enrollments:
    if a['account_key'] not in unique_engagements and a['join_date'] != a['cancel_date']:
        print(a)

{'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'is_canceled': True, 'days_to_cancel': 59, 'status': 'canceled', 'account_key': '1304', 'is_udacity': True}
{'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'is_canceled': True, 'days_to_cancel': 99, 'status': 'canceled', 'account_key': '1304', 'is_udacity': True}
{'cancel_date': None, 'join_date': datetime.datetime(2015, 2, 25, 0, 0), 'is_canceled': False, 'days_to_cancel': None, 'status': 'current', 'account_key': '1101', 'is_udacity': True}


## Tracking Down the Remaining Problems

In [10]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [11]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [12]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print(len(non_udacity_enrollments))
print(len(non_udacity_engagement))
print(len(non_udacity_submissions))

1622
135656
3634


## Refining the Question

In [25]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}

for a in non_udacity_enrollments:
    if (not a['is_canceled'] or a['days_to_cancel'] > 7) and (a['account_key'] not in paid_students or a['join_date'] > paid_students[a['account_key']]):
        paid_students[a['account_key']] = a['join_date']

print(len(paid_students))

995


## Getting Data from First Week

In [42]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7 and time_delta.days >=0

In [43]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_enrollments = [a for a in non_udacity_enrollments if a['account_key'] in paid_students.keys()]
paid_engagements = [a for a in non_udacity_engagement if a['account_key'] in paid_students.keys()]
paid_submissions = [a for a in non_udacity_submissions if a['account_key'] in paid_students.keys()]

paid_engagement_in_first_week =[a for a in paid_engagements if within_one_week(paid_students[a['account_key']], a['utc_date'])]

print(len(paid_engagement_in_first_week))

6919


## Exploring Student Engagement

In [58]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
def group_by_key(inList, inKey):
    recs_by_account = defaultdict(list)
    for rec in inList:
        recs_by_account[rec[inKey]].append(rec)
    return recs_by_account

engagement_by_account = group_by_key(paid_engagement_in_first_week, 'account_key')
'''
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)
'''

"\nengagement_by_account = defaultdict(list)\nfor engagement_record in paid_engagement_in_first_week:\n    account_key = engagement_record['account_key']\n    engagement_by_account[account_key].append(engagement_record)\n"

In [59]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)

def get_totals_by_acct_for_key(inDict, inKey):
    total_by_account = {}
    for key, rec in inDict.items():
        total = 0
        for subRec in rec:
            total += subRec[inKey]
        total_by_account[key] = total
    return total_by_account

total_minutes_by_account = get_totals_by_acct_for_key(engagement_by_account, 'total_minutes_visited')
'''
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes
'''

"\ntotal_minutes_by_account = {}\nfor account_key, engagement_for_student in engagement_by_account.items():\n    total_minutes = 0\n    for engagement_record in engagement_for_student:\n        total_minutes += engagement_record['total_minutes_visited']\n    total_minutes_by_account[account_key] = total_minutes\n"

In [60]:
import numpy as np
def get_statistics(inDict):
    total = list(inDict.values())
    print('Mean:', np.mean(total))
    print('Standard deviation:', np.std(total))
    print('Minimum:', np.min(total))
    print('Maximum:', np.max(total))
# Summarize the data about minutes spent in the classroom
get_statistics(total_minutes_by_account)

Mean: 306.708326753
Standard deviation: 412.996933409
Minimum: 0.0
Maximum: 3564.7332645


## Debugging Data Analysis Code

In [61]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

for key, val in total_minutes_by_account.items():
    if val == np.max(total_minutes):
        print(engagement_by_account[key])

[{'projects_completed': 0, 'utc_date': datetime.datetime(2015, 7, 9, 0, 0), 'total_minutes_visited': 850.519339666, 'lessons_completed': 4, 'account_key': '163', 'num_courses_visited': 4}, {'projects_completed': 0, 'utc_date': datetime.datetime(2015, 7, 10, 0, 0), 'total_minutes_visited': 872.633923334, 'lessons_completed': 6, 'account_key': '163', 'num_courses_visited': 6}, {'projects_completed': 0, 'utc_date': datetime.datetime(2015, 7, 11, 0, 0), 'total_minutes_visited': 777.018903666, 'lessons_completed': 6, 'account_key': '163', 'num_courses_visited': 2}, {'projects_completed': 0, 'utc_date': datetime.datetime(2015, 7, 12, 0, 0), 'total_minutes_visited': 294.568774, 'lessons_completed': 2, 'account_key': '163', 'num_courses_visited': 1}, {'projects_completed': 0, 'utc_date': datetime.datetime(2015, 7, 13, 0, 0), 'total_minutes_visited': 471.2139785, 'lessons_completed': 1, 'account_key': '163', 'num_courses_visited': 3}, {'projects_completed': 0, 'utc_date': datetime.datetime(2015

## Lessons Completed in First Week

In [62]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

total_lessons_by_account = get_totals_by_acct_for_key(engagement_by_account, 'lessons_completed')
get_statistics(total_lessons_by_account)

Mean: 1.63618090452
Standard deviation: 3.00256129983
Minimum: 0
Maximum: 36


## Number of Visits in First Week

In [ ]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.
for rec in paid_engagements

## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

passing_engagement =
non_passing_engagement =

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.